## Loading the Data
First we need to get the MNIST database onto our computer and put in the right matrix format. The data was acquired from the cvdfoundation GitHub repository.

In [220]:
import numpy as np
import struct

def load_images(filename):
    with open(filename, "rb") as f: # autocloses the file after reading vs:f = open(...), data = f.read(), f.close()
        _, num, rows, cols = struct.unpack(">IIII", f.read(16)) # Read 16 bytes of metadata, put 4 in each, "_" is a throwaway magic number in the dataset 
        data = np.frombuffer(f.read(), dtype=np.uint8) # read pixel data as byte-values, iterates automatically through file
        return data.reshape(num, rows, cols) # reshape to (num_samples, 28, 28)

def load_labels(filename):
    with open(filename, "rb") as f:
        _, num = struct.unpack(">II", f.read(8)) # Read metadata from labels file
        data = np.frombuffer(f.read(), dtype=np.uint8)
        return data


### Explaining What's Going on

Data is a numpy ndarray in this case, which is a multidimensional array. After we load all of the bytes in a flattened 1D array, we use the reshape function to create a 3D array where there are 60,000 cells of 28 * 28 pixel images. 

The data in the next function with the labels doesn't need to be reshapen because it's supposed to be a 1D array with only byte values for each integer from 0 to 9.

In [ ]:
x_train = load_images("train-images.idx3-ubyte")
y_train = load_labels("train-labels.idx1-ubyte")

x_test = load_images("t10k-images.idx3-ubyte")
y_test = load_labels("t10k-labels.idx1-ubyte")

print("MNIST dataset successfully loaded into numpy arrays.")

: 

Because of the common function notation y = f(x), the input data is named x while the output labels are named y.

## Preprocessing the data

### Flattening the image array
The next step is to flatten the array back into a 784 dimensional vector and normalize the values of each pixel.

In [222]:
x_train = x_train.reshape(x_train.shape[0], 28 * 28)
x_test = x_test.reshape(x_test.shape[0], 28 * 28)

# We test to make sure the arrays are the right shape now
print(x_train.shape)
print(x_test.shape)

(60000, 784)
(10000, 784)


### Normalizing the array values
Next we need to normalize all of the values in the array so that they are between 0 and 1 instead of 0 to 255. This isn't always done but we do it on this perceptron so we can measure a neuron's activation more easily.

In [223]:
x_train = x_train.astype(np.float32) / 255.0
x_test = x_test.astype(np.float32) / 255.0

# Testing to see that it's properly normalized
print(x_train.min(), x_train.max())  # Should print: 0.0 1.0
print(x_test.min(), x_test.max())    # Should print: 0.0 1.0

0.0 1.0
0.0 1.0


This typecasts the uint8 values from the array into float32. Now we can normalize this by dividing it by the max value - 255. Now every value will be a ratio compared to 255, leaving a value from 0 to 1.

### One-hot Encoding the labels
After flattening the arrays, we need to one-hot encode each digit. 

0 -> 1000000000

1 -> 0100000000

2 -> 0010000000

...and so on. We do this because we only want one output neuron firing for each digit in this case.

In [224]:
def one_hot_encode(y, num_classes=10):
    return np.eye(num_classes)[y]

y_train = one_hot_encode(y_train)
y_test = one_hot_encode(y_test)

# Verify the shape of our array and its formatting
print(y_train.shape)
print(y_test.shape)

print(y_test[0])

(60000, 10)
(10000, 10)
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


## Defining the Network Architecture

Now that all of the preprocessing steps are done, we need to initialize the neural network. In this case, we will make one with two hidden layers consisting of 128 neurons, an input layer with 784 neurons, and an output with 10 neurons - one for each digit. 

In [225]:
# Define the sizes of the layers

input_size = 784
hidden_size1 = 128
hidden_size2 = 64
output_size = 10

#Initialize the weights and biases with random values
np.random.seed(42)

W1 = np.random.randn(input_size, hidden_size1) * 0.01 # (784, 128)
b1 = np.zeros((1, hidden_size1)) 

W2 = np.random.randn(hidden_size1, hidden_size2) * 0.01
b2 = np.zeros((1, hidden_size2))

W3 = np.random.randn(hidden_size2, output_size) * 0.01
b3 = np.zeros((1, output_size))

# Verify everything
print("W1 shape:", W1.shape)  # (784, 128)
print("b1 shape:", b1.shape)  # (1, 128)

print("W2 shape:", W2.shape)  # (128, 64)
print("b2 shape:", b2.shape)  # (1, 64)

print("W3 shape:", W3.shape)  # (64, 10)
print("b3 shape:", b3.shape)  # (1, 10)

W1 shape: (784, 128)
b1 shape: (1, 128)
W2 shape: (128, 64)
b2 shape: (1, 64)
W3 shape: (64, 10)
b3 shape: (1, 10)


## Forward Propagation

With all the weights randomly initialized, we now do a forward pass of our neural network in its current state and see what we get.

Forward propagation is just the process of passing the input at each payer through the weights and biases to get the outputs. 

This can be described with the function Z = W*X + b

In [226]:
# Defining activation functions for our layers

def relu(z): # Non linear activation function for hidden layers
    return np.maximum(0, z)

def softmax(z): # Used for the output layer to turn the outputs into a probability from 0 to 1
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)


#### Resources to find out more about these activation functions:

[reLU](https://en.wikipedia.org/wiki/Rectifier_(neural_networks))

[Softmax](https://en.wikipedia.org/wiki/Softmax_function)

In [227]:
def forward_propagation(x):
    global W1, b1, W2, b2, W3, b3 # Use the globally defined values for these variables

    # Pass through 1st hidden layer
    Z1 = np.dot(x, W1) + b1 # Pass input layer through weights -> take the dot product of both matrices then add bias
    A1 = relu(Z1) # Pass the resulting matrix through activation function

    # Do the same for the 2nd hidden layer
    Z2 = np.dot(A1, W2) + b2
    A2 = relu(Z2)

    # Output layer

    Z3 = np.dot(A2, W3) + b3
    A3 = softmax(Z3) # The output uses a different activation function

    return Z1, A1, Z2, A2, Z3, A3 # Return all intermediate values for backpropagation

# Running forward propagation:

batch_X = x_train[:5]  # Take first 5 images
_, _, _, _, _, predictions = forward_propagation(batch_X)

print("Predictions (Softmax Output):\n", predictions)
print("Sum of each row (should be 1):", np.sum(predictions, axis=1))  # Verify softmax sums to 1

Predictions (Softmax Output):
 [[0.09999784 0.1000113  0.09999249 0.10004964 0.09998993 0.10003599
  0.09999102 0.09996785 0.09998571 0.09997824]
 [0.09992773 0.1000102  0.1000791  0.10006082 0.09997785 0.10000442
  0.10006154 0.10000583 0.0999256  0.0999469 ]
 [0.10004008 0.09996258 0.10000727 0.10001555 0.10002374 0.09999946
  0.10001566 0.10000466 0.099969   0.09996199]
 [0.09998447 0.09997845 0.10005079 0.10003316 0.10002571 0.09999959
  0.09999116 0.10000886 0.09996861 0.09995919]
 [0.10003088 0.09999359 0.09996209 0.10001275 0.10003859 0.10001534
  0.09995681 0.10003345 0.09996493 0.09999156]]
Sum of each row (should be 1): [1. 1. 1. 1. 1.]


## Computing the Loss Function

We use the labels for the dataset to compute a loss function to quantize how off we were. We're computing the cross entropy loss in this case, find more info [here](https://en.wikipedia.org/wiki/Cross-entropy).

In [228]:
def compute_loss(y_true, y_pred):
    m = y_true.shape[0] # number of samples in batch
    loss = -np.sum(y_true * np.log(y_pred + 1e-8)) / m # Add epsilon to avoid log(0)
    return loss

# Running the loss function

batch_X = x_train[:5]  # First 5 images
batch_Y = y_train[:5]  # First 5 labels

_, _, _, _, _, predictions = forward_propagation(batch_X)  # Get predicted probabilities

loss = compute_loss(batch_Y, predictions)
print("Cross-Entropy Loss:", loss)


Cross-Entropy Loss: 2.302670109823364


## Backpropagation
Backpropagation is how you compute gradients to see how much each weight affected the output, and you use this to update the weights. 

Computing the gradients involves basic partial derivatives and chain rule, and more information on the math can be found [here](https://en.wikipedia.org/wiki/Backpropagation).

We are trying to calculate the gradient of the Loss function with respect to W.

Keep in mind that
$$
Loss = L(A(Z(W)))
$$

so calculating the gradient for weights involves simple chain rule:

$$
\frac{\partial L}{\partial W} = \frac{\partial L}{\partial A} \cdot \frac{\partial A}{\partial Z} \cdot \frac{\partial Z}{\partial W}
$$

Where L is the loss function, A is the activation function of a layer, Z is the linear transformation Z = Wx + b, and W is the weight. Everything is computed with respect to W basically.

Backpropagation for biases: 
$$
\frac{\partial Z}{\partial b} = 1 
$$
because biases don't depend on the input like Wx does.

Thus, the gradient of the biases is:
$$
\frac{\partial L}{\partial b} = \frac{\partial L}{\partial Z}
$$



In [229]:
def relu_derivative(Z):
    """Derivative of ReLU function"""
    return Z > 0  # Returns 1 for positive values, 0 otherwise

def backward_propagation(X, Y, Z1, A1, Z2, A2, Z3, A3):
    """
    Computes the gradients using backpropagation.
    X: Input data
    Y: True labels (one-hot encoded)
    Z1, A1, Z2, A2, Z3, A3: Values from forward propagation
    """
    global W1, b1, W2, b2, W3, b3  # Use global weights and biases
    
    m = X.shape[0]  # Batch size

    # Compute gradient for Output Layer (Softmax & Cross-Entropy Loss)
    dZ3 = A3 - Y  # Gradient of loss with respect to Z3
    dW3 = np.dot(A2.T, dZ3) / m  # Gradient of loss w.r.t W3
    db3 = np.sum(dZ3, axis=0, keepdims=True) / m  # Gradient of loss w.r.t b3

    # Compute gradient for Hidden Layer 2 (ReLU)
    dA2 = np.dot(dZ3, W3.T)  # Backpropagate through W3
    dZ2 = dA2 * relu_derivative(Z2)  # Apply ReLU derivative
    dW2 = np.dot(A1.T, dZ2) / m  # Gradient of loss w.r.t W2
    db2 = np.sum(dZ2, axis=0, keepdims=True) / m  # Gradient of loss w.r.t b2

    # Compute gradient for Hidden Layer 1 (ReLU)
    dA1 = np.dot(dZ2, W2.T)  # Backpropagate through W2
    dZ1 = dA1 * relu_derivative(Z1)  # Apply ReLU derivative
    dW1 = np.dot(X.T, dZ1) / m  # Gradient of loss w.r.t W1
    db1 = np.sum(dZ1, axis=0, keepdims=True) / m  # Gradient of loss w.r.t b1

    return dW1, db1, dW2, db2, dW3, db3

Now we're going to test our backpropagation function:

In [230]:
batch_X = x_train[:5]  # First 5 images
batch_Y = y_train[:5]  # First 5 labels

# Forward propagation
Z1, A1, Z2, A2, Z3, A3 = forward_propagation(batch_X)

# Compute gradients
dW1, db1, dW2, db2, dW3, db3 = backward_propagation(batch_X, batch_Y, Z1, A1, Z2, A2, Z3, A3)

# Print gradient shapes to verify
print("dW1 shape:", dW1.shape)  # Expected: (784, 128)
print("db1 shape:", db1.shape)  # Expected: (1, 128)
print("dW2 shape:", dW2.shape)  # Expected: (128, 64)
print("db2 shape:", db2.shape)  # Expected: (1, 64)
print("dW3 shape:", dW3.shape)  # Expected: (64, 10)
print("db3 shape:", db3.shape)  # Expected: (1, 10)

dW1 shape: (784, 128)
db1 shape: (1, 128)
dW2 shape: (128, 64)
db2 shape: (1, 64)
dW3 shape: (64, 10)
db3 shape: (1, 10)


### Update Weights

Now we use the computed gradients to nudge the weights using gradient descent.

The gradient descent algorithm looks like this:

$$
{W} = {W} - {\alpha}\frac{\partial L}{\partial W}
$$

and we do the same for the biases:
$$
{b} = {b} - {\alpha}\frac{\partial L}{\partial b}
$$

Where alpha is the learning rate, or how big the increments to W are. 


In [231]:
def update_parameters(dW1, db1, dW2, db2, dW3, db3, learning_rate=0.01):
    """
    Updates weights and biases using gradient descent.
    dW1, db1, dW2, db2, dW3, db3: Gradients from backpropagation
    learning_rate: Step size for updates
    """
    global W1, b1, W2, b2, W3, b3  # Access global weights and biases

    # Update weights and biases using gradient descent
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    W3 -= learning_rate * dW3
    b3 -= learning_rate * db3

Now we test this code:

In [232]:
# Take a batch of 5 images
batch_X = x_train[:5]
batch_Y = y_train[:5]

# Forward propagation
Z1, A1, Z2, A2, Z3, A3 = forward_propagation(batch_X)

# Compute gradients
dW1, db1, dW2, db2, dW3, db3 = backward_propagation(batch_X, batch_Y, Z1, A1, Z2, A2, Z3, A3)

# Store original W1 for comparison
W1_before = W1.copy()

# Update weights
update_parameters(dW1, db1, dW2, db2, dW3, db3, learning_rate=0.01)

# Compare before and after update
print("W1 change:", np.sum(np.abs(W1 - W1_before)))  # Should be non-zero if weights updated


W1 change: 0.020560581875656167


### Training for multiple epochs

Now that all of our functions are working, the next step is to actually train the model for multiple epochs and go through all of the data. 

An epoch is one full pass through the data set, and we will be training the model for multiple epochs.

For each epoch, we:

1. Loop through all of our mini-batches of data 
2. Perform forward propagation and compute predictions
3. Compute the loss
4. Perform backpropagation
5. Update all of the weights using gradient descent

And then we repeat this for multiple epochs.

In [233]:
def train_model(x_train, y_train, epochs=10, batch_size=32, learning_rate=0.01):
    """
    Trains the neural network using mini-batch gradient descent.
    
    x_train: Training images (flattened)
    y_train: One-hot encoded labels
    epochs: Number of times the model sees the full dataset
    batch_size: Number of samples processed at a time
    learning_rate: Step size for weight updates
    """
    num_samples = x_train.shape[0]  # Number of training samples
    num_batches = num_samples // batch_size  # Total batches per epoch
    
    for epoch in range(epochs):
        total_loss = 0  # Track total loss for the epoch
        
        # Shuffle the training data so the model doesn't learn patterns based on the order of the data
        indices = np.random.permutation(num_samples)
        x_train_shuffled = x_train[indices]
        y_train_shuffled = y_train[indices]

        # Process mini-batches
        for i in range(0, num_samples, batch_size):
            batch_X = x_train_shuffled[i:i + batch_size]
            batch_Y = y_train_shuffled[i:i + batch_size]

            # Forward propagation
            Z1, A1, Z2, A2, Z3, A3 = forward_propagation(batch_X)

            # Compute loss
            loss = compute_loss(batch_Y, A3)
            total_loss += loss  # Accumulate loss over all batches

            # Backpropagation
            dW1, db1, dW2, db2, dW3, db3 = backward_propagation(batch_X, batch_Y, Z1, A1, Z2, A2, Z3, A3)

            # Update parameters
            update_parameters(dW1, db1, dW2, db2, dW3, db3, learning_rate)

        # Print progress at the end of each epoch
        avg_loss = total_loss / num_batches
        print(f"Epoch {epoch + 1}/{epochs} - Loss: {avg_loss:.4f}")


Now we can go ahead and train the model on all of the data:

In [234]:
train_model(x_train, y_train, epochs=10, batch_size=32, learning_rate=0.01)

Epoch 1/10 - Loss: 2.3009
Epoch 2/10 - Loss: 2.1733
Epoch 3/10 - Loss: 0.9216
Epoch 4/10 - Loss: 0.5284
Epoch 5/10 - Loss: 0.4272
Epoch 6/10 - Loss: 0.3774
Epoch 7/10 - Loss: 0.3350
Epoch 8/10 - Loss: 0.2944
Epoch 9/10 - Loss: 0.2592
Epoch 10/10 - Loss: 0.2304


### Evaluating the model

Now that everything has been trained, we need to evaluate it on new data that it hasn't seen before.

In [235]:
def evaluate_model(x_test, y_test):
    """
    Evaluates the trained model on the test dataset.
    
    x_test: Test images (flattened)
    y_test: True labels (one-hot encoded)
    """
    # Forward propagation on the entire test set
    _, _, _, _, _, predictions = forward_propagation(x_test)

    # Compute test loss
    test_loss = compute_loss(y_test, predictions)

    # Convert softmax outputs to class predictions
    predicted_labels = np.argmax(predictions, axis=1)
    true_labels = np.argmax(y_test, axis=1)

    # Compute accuracy
    accuracy = np.mean(predicted_labels == true_labels) * 100

    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Accuracy: {accuracy:.2f}%")
    
    return accuracy

test_accuracy = evaluate_model(x_test, y_test)


Test Loss: 0.2150
Test Accuracy: 93.82%


### Hyperparamter Tuning

Now that we have all the code to train and evaluate the model, we can try to tune the parameters to get better accuracy.

There are a few strategies we could use:

1. Manual Tuning - Pretty much just trial and error
2. Grid Search - Try various combinations of parameters (Ex: Train models with learning_rate = [0.01, 0.001, 0.0001] and hidden_neurons = [64, 128, 256].)
3. Random search: Randomly change parameters instead of trying all combinations

Below, we make a function to test different hyperparameter values.

In [236]:
def train_and_evaluate(learning_rate=0.01, hidden_size1=128, hidden_size2=64, batch_size=32, epochs=10):
    """
    Trains and evaluates the model with different hyperparameters.
    
    learning_rate: Step size for weight updates
    hidden_size1: Number of neurons in first hidden layer
    hidden_size2: Number of neurons in second hidden layer
    batch_size: Number of samples processed at a time
    epochs: Number of times the model sees the full dataset
    """
    global W1, b1, W2, b2, W3, b3

    # Print Hyperparameters
    print("\n=================================")
    print("Training with Hyperparameters:")
    print(f"  Learning Rate: {learning_rate}")
    print(f"  Hidden Layer 1 Neurons: {hidden_size1}")
    print(f"  Hidden Layer 2 Neurons: {hidden_size2}")
    print(f"  Batch Size: {batch_size}")
    print(f"  Epochs: {epochs}")
    print("=================================")

    # Reinitialize Weights with New Hidden Layer Sizes
    input_size = 784  # 28x28 pixels
    output_size = 10  # 10 classes

    np.random.seed(42)
    W1 = np.random.randn(input_size, hidden_size1) * 0.01
    b1 = np.zeros((1, hidden_size1))
    W2 = np.random.randn(hidden_size1, hidden_size2) * 0.01
    b2 = np.zeros((1, hidden_size2))
    W3 = np.random.randn(hidden_size2, output_size) * 0.01
    b3 = np.zeros((1, output_size))

    # Train the Model
    train_model(x_train, y_train, epochs, batch_size, learning_rate)

    # Evaluate Model
    test_accuracy = evaluate_model(x_test, y_test)

    # Print Final Accuracy
    print(f"Final Test Accuracy: {test_accuracy:.2f}%")
    return test_accuracy


Now lets try some parameters randomly:

In [237]:
# Experiment 1: Higher Learning Rate
train_and_evaluate(learning_rate=0.1, hidden_size1=128, hidden_size2=64, batch_size=32, epochs=10)

# Experiment 2: Lower Learning Rate
train_and_evaluate(learning_rate=0.001, hidden_size1=128, hidden_size2=64, batch_size=32, epochs=10)

# Experiment 3: More Hidden Neurons
train_and_evaluate(learning_rate=0.01, hidden_size1=256, hidden_size2=128, batch_size=32, epochs=10)

# Experiment 4: Larger Batch Size
train_and_evaluate(learning_rate=0.01, hidden_size1=128, hidden_size2=64, batch_size=128, epochs=10)


Training with Hyperparameters:
  Learning Rate: 0.1
  Hidden Layer 1 Neurons: 128
  Hidden Layer 2 Neurons: 64
  Batch Size: 32
  Epochs: 10
Epoch 1/10 - Loss: 0.8316
Epoch 2/10 - Loss: 0.1798
Epoch 3/10 - Loss: 0.1163
Epoch 4/10 - Loss: 0.0883
Epoch 5/10 - Loss: 0.0713
Epoch 6/10 - Loss: 0.0585
Epoch 7/10 - Loss: 0.0483
Epoch 8/10 - Loss: 0.0400
Epoch 9/10 - Loss: 0.0334
Epoch 10/10 - Loss: 0.0288
Test Loss: 0.1036
Test Accuracy: 97.09%
Final Test Accuracy: 97.09%

Training with Hyperparameters:
  Learning Rate: 0.001
  Hidden Layer 1 Neurons: 128
  Hidden Layer 2 Neurons: 64
  Batch Size: 32
  Epochs: 10
Epoch 1/10 - Loss: 2.3024
Epoch 2/10 - Loss: 2.3019
Epoch 3/10 - Loss: 2.3015
Epoch 4/10 - Loss: 2.3012
Epoch 5/10 - Loss: 2.3009
Epoch 6/10 - Loss: 2.3007
Epoch 7/10 - Loss: 2.3004
Epoch 8/10 - Loss: 2.3001
Epoch 9/10 - Loss: 2.2996
Epoch 10/10 - Loss: 2.2991
Test Loss: 2.2985
Test Accuracy: 11.35%
Final Test Accuracy: 11.35%

Training with Hyperparameters:
  Learning Rate: 0.01
  

np.float64(75.48)

### Advanced methods to improve accuracy

There are some other methods to improve accuracy further. These include:

1. Regularization - if the model does well on training data but not on test data, meaning that it's likely overfitting

2. Dropout - randomly disabling neurons during training to prevent the model from relying too much on a subset of neurons

3. Other optimization techniques to replace gradient descent such as [momentum](https://www.youtube.com/watch?v=k8fTYJPd3_I&ab_channel=DeepLearningAI) and the [Adam Optimizer](https://www.geeksforgeeks.org/adam-optimizer/) (used by default in torch and tensorflow)

4. More advanced architectures such as CNNs

5. Data augmentation - creating more data samples simply by modifying existing data, such as by transforming the image date through scaling, rotation, etc.

6. Transfer learning - use a pretrained model and replace a few layers to adapt it to your application

7. Model ensemble - Train multiple models with different architectures and average their predictions